In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
CustomerID = np.arange(0, 9)
Spend = np.random.uniform(low=0, high=100, size=9)
data = pd.DataFrame({'CustomerID':CustomerID, 'Spend':Spend},
    index=range(9))

# Calculate percentiles
data['Spend_Quartile'] = pd.qcut(data['Spend'], q=4, 
    labels=range(1, 5))
data.sort_values('Spend', inplace=True)
data.reset_index(inplace=True, drop=True)
data

,CustomerID,Spend,Spend_Quartile
0,8,5.235064,1
1,1,15.750535,1
2,4,23.341597,1
3,0,25.845030,2
4,7,27.297191,2
5,6,33.503085,3
6,2,40.179699,3
7,3,47.174648,4
8,5,60.429661,4


In [3]:
CustomerID = np.arange(0, 9)
Recency_Days = np.random.randint(low=0, high=100, size=9)
data = pd.DataFrame({'CustomerID':CustomerID, 
    'Recency_Days':Recency_Days}, index=range(9))

# Divide into groups based on quartiles
data['Recency_Quartile'] = pd.qcut(data['Recency_Days'], q=4,
    labels=range(4, 0, -1))
data.sort_values('Recency_Days', inplace=True)
data.reset_index(inplace=True, drop=True)
data

,CustomerID,Recency_Days,Recency_Quartile
0,2,8,4
1,5,33,4
2,1,48,4
3,3,50,3
4,7,62,3
5,4,68,2
6,6,68,2
7,0,69,1
8,8,93,1


In [4]:
CustomerID = np.arange(0, 9)
Recency_Days = np.random.randint(low=0, high=100, size=9)
data = pd.DataFrame({'CustomerID':CustomerID, 
    'Recency_Days':Recency_Days}, index=range(9))

# Create string labels
r_labels = ['Active', 'Lapsed', 'Inactive', 'Churned']
# Divide into groups based on quartiles
data['Recency_Quartile'] = pd.qcut(data['Recency_Days'], q=4,
    labels=r_labels)
data.sort_values('Recency_Days', inplace=True)
data.reset_index(inplace=True, drop=True)
data

,CustomerID,Recency_Days,Recency_Quartile
0,1,5,Active
1,7,19,Active
2,3,40,Active
3,6,41,Lapsed
4,2,45,Lapsed
5,5,63,Inactive
6,4,68,Inactive
7,8,81,Churned
8,0,99,Churned


In [5]:
# Dataset & preparations
online = pd.read_csv("../data/online.csv", index_col=[0], 
    parse_dates=['InvoiceDate'])
online['InvoiceDate'] = online['InvoiceDate'].dt.date
online['TotalSum'] = online['Quantity'] * online['UnitPrice']
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,United Kingdom,12.60
482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,United Kingdom,1.45
263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,United Kingdom,22.50
495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,United Kingdom,2.10
204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,United Kingdom,17.85


In [6]:
# Create a hypothetical snapshot day data
print(f"Min:{min(online.InvoiceDate)}; Max:{max(online.InvoiceDate)}")
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days=1)
snapshot_date

Min:2010-12-01; Max:2011-12-09


datetime.date(2011, 12, 10)

In [7]:
# Calculate RFM metrics
# Aggregate data on a customer level
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})
# Rename columns for easier interpretation
datamart.rename(columns = {
    'InvoiceDate': 'Recency', 
    'InvoiceNo': 'Frequency',
    'TotalSum': 'MonetaryValue'}, inplace=True)
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,3,27,992.82
12748,1,967,7522.06
12749,4,37,813.45
12820,4,17,268.02
12822,71,9,146.15


In [8]:
# Recency quartile
r_labels = range(4, 0, -1)
r_quartiles = pd.qcut(datamart['Recency'], q=4, labels=r_labels)

# Frequency quartile
f_labels = range(1, 5)
f_quartiles = pd.qcut(datamart['Frequency'], q=4, labels=f_labels)

# Monetary quartile
m_labels = range(1, 5)
m_quartiles = pd.qcut(datamart['MonetaryValue'], q=4, labels=m_labels)
datamart = datamart.assign(
    R=r_quartiles.values,
    F=f_quartiles.values,
    M=m_quartiles.values)

datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747,3,27,992.82,4,4,4
12748,1,967,7522.06,4,4,4
12749,4,37,813.45,4,4,4
12820,4,17,268.02,4,3,3
12822,71,9,146.15,2,2,3


In [9]:
# Build RFM segment & RFM score
# Concatenate RFM quartile values to RFM_Segment
def join_rfm(x):
    r = str(x['R'].astype(int))
    f = str(x['F'].astype(int))
    m = str(x['M'].astype(int))
    return r + f + m

datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)
# Sum RFM quartiles values to RFM_Score
datamart['RFM_Score'] = datamart[['R', 'F', 'M']].sum(axis=1)
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,3,27,992.82,4,4,4,444,12
12748,1,967,7522.06,4,4,4,444,12
12749,4,37,813.45,4,4,4,444,12
12820,4,17,268.02,4,3,3,433,10
12822,71,9,146.15,2,2,3,223,7


In [10]:
# Top 10 largest RFM segments
datamart.groupby('RFM_Segment').size()\
    .sort_values(ascending=False)[:10]

RFM_Segment
444    379
111    345
344    165
211    165
233    131
222    130
311    128
333    124
122    116
433    114
dtype: int64

In [11]:
# Filtering on RFM segments
datamart[datamart['RFM_Segment']=='111'][:5]

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12837,174,2,10.55,1,1,1,111,3
12852,295,2,32.55,1,1,1,111,3
12902,265,4,42.03,1,1,1,111,3
12922,162,4,57.24,1,1,1,111,3
12929,312,3,42.90,1,1,1,111,3


In [14]:
# Summary metrics per RFM score
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']})\
    .round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3           257.6       2.0          28.6   345
4           177.8       3.2          48.0   343
5           144.2       4.3          78.6   414
6           105.8       6.4         149.2   434
7            83.3       8.9         163.7   385
8            63.8      12.9         196.2   381
9            49.1      16.7         330.9   348
10           33.6      24.8         442.7   363
11           22.0      39.2         715.1   309
12            8.1      77.2        1715.5   379

In [15]:
# Grouping into named segments
# Use RFM score to group customers into Gold, Silver & Bronze segments
def segment_me(df):
    if df['RFM_Score'] >= 9:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'
    
datamart['General_Segment'] = datamart.apply(segment_me, axis=1)
datamart.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']})\
    .round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            217.8       2.6          38.3   688
Gold               28.0      40.2         819.9  1399
Silver            100.4       8.0         145.6  1614

In [17]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'Top'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 10)):
        return 'Middle'
    else:
        return 'Low'

# Create a new variable RFM_Level
datamart['RFM_Level'] = datamart.apply(rfm_level, axis=1)

# Print the header with top 5 rows to the console
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment,RFM_Level
CustomerID,,,,,,,,,,
12747,3,27,992.82,4,4,4,444,12,Gold,Top
12748,1,967,7522.06,4,4,4,444,12,Gold,Top
12749,4,37,813.45,4,4,4,444,12,Gold,Top
12820,4,17,268.02,4,3,3,433,10,Gold,Top
12822,71,9,146.15,2,2,3,223,7,Silver,Middle


In [18]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = datamart.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
  
  	# Return the size of each segment
    'MonetaryValue': ['mean', 'count']
}).round(1)

# Print the aggregated dataset
rfm_level_agg

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Level                                      
Low         190.1       3.2          53.4  1102
Middle       77.1      10.9         205.2  1548
Top          21.0      47.9         981.8  1051